In [59]:
import pandas as pd

df1 = pd.DataFrame({'snippet': ['China North Industries Group Corp Ltd', 'Aviation Industry Corp of China Ltd',
                                'Lockheed Martin Corp', 'Raytheon Technologies Corp', 'The Boeing Co',
                                'Airbus SE', 'Hanwha Corp', 'China Aerospace Science and Technology Corp',
                                'ThyssenKrupp AG', 'General Dynamics Corp'],
                    'tooltip': ['Defense']*10})
df2 = pd.DataFrame({'snippet': ['Cargill Inc', 'BASF SE', 'Archer Daniels Midland Co',
                                'Wilmar International Ltd', 'Bunge Ltd', 'Bayer AG',
                                'CHS Inc', 'Nutrien Ltd', 'BayWa AG', 'Associated British Foods Plc'],
                    'tooltip': ['Agricultural products']*10})

df3 = pd.DataFrame({'snippet': ['Volkswagen AG', 'Toyota Motor Corp', 'Stellantis NV', 'Mercedes-Benz Group AG', 'Ford Motor Co', 'Bayerische Motoren Werke AG', 'General Motors Co', 'Honda Motor Co Ltd', 'SAIC Motor Corp Ltd', 'Hyundai Motor Co'],
                    'tooltip': ['Automotive']*10})

df4 = pd.DataFrame({'snippet': ['Deloitte Touche Tohmatsu Ltd', 'PricewaterhouseCoopers International Ltd', 'Ernst & Young Global Ltd', 'KPMG International Coop', 'Marsh & McLennan Companies Inc', 'Aon Plc', 'BDO International Ltd', 'The Boston Consulting Group', 'Booz Allen Hamilton Holding Corp', 'Grant Thornton International Ltd'],
                    'tooltip': ['Consulting']*10})
df5 = pd.DataFrame({'snippet': ["BASF SE", "Dow Inc", "Saudi Basic Industries Corp", "LyondellBasell Industries NV", "Mitsubishi Chemical Group Corp", "3M Co", "Linde plc", "Air Liquide SA", "Sumitomo Chemical Co Ltd", "Henkel AG & Co KGaA"],
                    'tooltip': ['Chemical']*10})
df6 = pd.DataFrame({'snippet': ['China Railway Engineering Group Co Ltd', 'China Railway Construction Corp Ltd', 'China Railway Construction Corporation Group', 'China State Construction Engineering Corp Ltd', 'China Communications Construction Group Ltd', 'Power Construction Corporation of China', 'Actividades de Construccion y Servicios SA', 'Larsen & Toubro Ltd', 'China National Chemical Engineering Co Ltd', 'Vinci SA'],
                    'tooltip': ['Construction']*10})
df7 = pd.DataFrame({'snippet': ['The Coca-Cola Co', 'PepsiCo Inc', 'Anheuser-Busch InBev NV/SA', 'Nestle SA', 'Unilever Plc', 'The Procter & Gamble Co', 'Heineken NV', 'Mars Inc', 'Mondelez International Inc', 'Danone SA'],
                    'tooltip': ['Consumer Packaged Goods']*10})
df8 = pd.DataFrame({'snippet': ['Industrial and Commercial Bank of China Ltd', 'China Construction Bank Corp', 'Agricultural Bank of China Ltd', 'Bank of China Ltd', 'JPMorgan Chase & Co', 'Credit Agricole SA', 'Bank of America Corp', 'BNP Paribas SA', 'Wells Fargo & Co', 'Postal Savings Bank of China Co Ltd'],
                    'tooltip': ['Banking']*10})


# concatenate dataframes
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], ignore_index=True)
def add_suffix(name):
    return name + " conducted a survey about sustainable supply chains with 525 large corporations across Argentina, Brazil, Canada, Mexico and the United States"

# apply the function to the column and assign the result to a new column
df['snippet'] = df['snippet'].apply(add_suffix)

snippets = list(df['snippet'])
df.to_csv("companies_tooltip.csv", index=False)

# SBERT

In [61]:
from sentence_transformers import SentenceTransformer
import pickle

model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

sbert_dict = {}
for name in df['snippet']:
    sbert_dict[name] = model.encode(name)

with open('labeled_companies_sbert.pkl', 'wb') as f:
    pickle.dump(sbert_dict, f)

In [ ]:
import gensim.downloader as api
import numpy as np
import pandas as pd 

#Load Pretrained Glove Embedding
glove_dim = 300
glove_wiki = api.load(f"glove-wiki-gigaword-{glove_dim}")

In [ ]:
def get_GloVe(text, size, vectors, aggregation='mean'):
    vec = np.zeros(size).reshape((glove_dim,)) # create for size of glove embedding and assign all values 0
    count = 0
    for word in text.split():
        try:
            vec += vectors[word].reshape((glove_dim,)) # update vector with new word
            count += 1 # counts every word in sentence
        except KeyError:
            continue
    if aggregation == 'mean':
        if count != 0:
            vec /= count  #get average of vector to create embedding for sentence
        return vec
    elif aggregation == 'sum':
        return vec

glove_dict = {}
for sent in df['snippet']:
    glove_dict[sent] = get_GloVe(sent, glove_dim, glove_wiki) #.reshape(dim,)

with open('labeled_companies_glove.pkl', 'wb') as f:
    pickle.dump(glove_dict, f)

In [ ]:
list(glove_dict.values())[0].reshape
type(list(sbert_dict.values())[0])

numpy.ndarray

In [ ]:
# !pip install laserembeddings
# !python -m laserembeddings download-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import argparse
import json
import os
import torch

from laserembeddings import Laser

laser = Laser()
sentences = list(df['snippet'])
# print(sentences)
_sent_embs = laser.embed_sentences(sentences, lang='en')
laser_dict = {}
for sent, emb in zip(sentences, _sent_embs):
    laser_dict[sent] = emb

with open('labeled_companies_laser.pkl', 'wb') as f:
    pickle.dump(laser_dict, f)

# _sent_tensors = [torch.from_numpy(j) for j in _sent_embs]
# torch.stack(_sent_tensors)

In [58]:
embeddings_st = encode_skipthoughts(df['snippet'])

torch.Size([80, 27])


tensor([[ 0.0705, -0.1086,  0.1890,  ..., -0.0368,  0.0247, -0.1964],
        [ 0.0767, -0.1767,  0.1797,  ..., -0.0287,  0.0301, -0.1958],
        [ 0.0691, -0.1769,  0.2703,  ..., -0.0286,  0.0315, -0.2011],
        ...,
        [ 0.0758, -0.1185,  0.2652,  ..., -0.0218,  0.0328, -0.2005],
        [ 0.0719, -0.1806,  0.2717,  ..., -0.0279,  0.0338, -0.1991],
        [ 0.0701, -0.1740,  0.1453,  ..., -0.0407,  0.0420, -0.1871]],
       grad_fn=<ViewBackward0>)

In [67]:
from utils import encode_skipthoughts
import pickle
sentences = list(df['snippet'])
embeddings_st = encode_skipthoughts(sentences)
st_dict = {}
for sent, emb in zip(sentences, embeddings_st):
    st_dict[sent] = emb.tolist()

with open('labeled_companies_skipthoughts.pkl', 'wb') as f:
    pickle.dump(st_dict, f)


torch.Size([80, 27])


In [ ]:
sbert_dict

In [68]:
type(st_dict['China North Industries Group Corp Ltd conducted a survey about sustainable supply chains with 525 large corporations across Argentina, Brazil, Canada, Mexico and the United States'])

list